In [ ]:
import syft as sy
sy.requires(">=0.8.1-beta")
sy.enable_autoreload()

In [ ]:
domain_1 = sy.orchestra.launch(name="test-domain-1", dev_mode=True, reset=True)

In [ ]:
domain_1_client = domain_1.login(email="info@openmined.org", password="changethis")

In [ ]:
domain_2 = sy.orchestra.launch(name="test-domain-2", dev_mode=True, reset=True)

In [ ]:
domain_2_client = domain_2.login(email="info@openmined.org", password="changethis")

In [ ]:
domain_1_client.exchange_route(domain_2_client)

In [ ]:
domain_1_client.domains

In [ ]:
assert len(domain_1_client.domains) == 1

In [ ]:
domain_2_client.domains

In [ ]:
assert len(domain_2_client.domains) == 1

In [ ]:
from syft.service.project.project import NewProjectSubmit
from syft.service.project.project import NewProject
from syft.service.project.project import DemocraticConsensusModel
from syft.service.metadata.node_metadata import NodeMetadata

In [ ]:
project = NewProjectSubmit(
    name="test project",
    description="my test project",
    shareholders=[domain_1_client.metadata.to(NodeMetadata).verify_key, domain_2_client.metadata.to(NodeMetadata).verify_key],
    consensus_model=DemocraticConsensusModel(),
)
project

In [ ]:
project = domain_1_client.api.services.project.start(project)

In [ ]:
assert isinstance(project, NewProject)

## Dataset Upload

In [ ]:
import numpy as np
dataset = sy.Dataset(name="Domains's... Private Data")
asset = sy.Asset(name="Secret data")

# Real Data
x = np.array([1, 2, 3])
asset.set_obj(x)

# Mock Data
y = np.array([1, 1, 1])
asset.set_mock(y, mock_is_real=False)
dataset.add_asset(asset)



In [ ]:
domain_1_client.upload_dataset(dataset)

In [ ]:
domain_1_client.datasets

In [ ]:
domain_1_dataset = domain_1_client.datasets[-1]
@sy.syft_function(
    input_policy=sy.ExactMatch(
        input_data=domain_1_dataset.assets[0]
    ),
    output_policy=sy.SingleExecutionExactOutput(),
)
def simple_function(input_data):

    return input_data+input_data

In [ ]:
simple_function

In [ ]:
req = domain_1_client.api.services.code.request_code_execution(simple_function)

In [ ]:
project.add_request(req)

In [ ]:
project

In [ ]:
for node in [domain_1, domain_2]:
    if node.node_type.value == "python":
        node.land()